## Multi-aperture telescope simulation sampler demonstration
Author: Ian Cunnyngham (Institute for Astronomy 2019,2020)

Simulates multi-aperture telescope optical performance with piston, tip, tilt actuation, atmospheres, etc. in HCIPy and returns samples in a format structured for deep learning applications

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import hcipy
from multi_aperture_psf import *

%matplotlib inline

ModuleNotFoundError: No module named 'hcipy'

### Set up sampler

Pick either a single large aperture or Mini-ELF mirror configuration

In [ ]:
# One big aperture with D=2.5m
nMir = 1
mir_centers = hcipy.CartesianGrid(np.array([[0], [0]]))
mir_diamater, pup_diamater = 2.5, 2.5

In [ ]:
# Mini-ELF: 2.5m ring of 15 D~=.5m apertures (grown slightly for closest packing)
nMir = 15
telescopeR = 1.25 # meters
mir_coords = hcipy.SeparatedCoords((np.array([telescopeR]), np.linspace(0, 2*np.pi, nMir+1)[:-1]))
mir_centers = hcipy.PolarGrid(mir_coords).as_('cartesian')
mir_diamater = np.sqrt((mir_centers.x[1]-mir_centers.x[0])**2 + (mir_centers.y[1]-mir_centers.y[0])**2) 
pup_diamater = max(mir_centers.x.max() - mir_centers.x.min(), mir_centers.y.max() - mir_centers.y.min()) + mir_diamater
pup_diamater *= 1.05  # Add a little extra for edges, not convinced not cutting off
print(mir_diamater, pup_diamater)

In [ ]:
mas_setup = {
    'mirror_config': {
        'positions': mir_centers,
        'aperture': hcipy.circular_aperture(mir_diamater),
        'pupil_extent': pup_diamater,
        'pupil_res': 256,
        'piston_scale': 1e-6,   # meters
        'tip_tilt_scale': 1e-6  # meters
    },
    # Two Filters, at 500nm and 1 micron, both with 4arcsec FOV and .05 frac bandwidth
    'filter_configs': [ {
        'central_lam': .5e-6,   # meters
        'focal_extent': 4,      # arcsec
        'focal_res': 256,
        'frac_bandwidth': .05,
        'num_samples': 3
    },{
        'central_lam': 1e-6,    # meters
        'focal_extent': 4,      # arcsec
        'focal_res': 256,
        'frac_bandwidth': .05,
        'num_samples': 3
    } ] 
}
mas_psf_sampler = MultiAperturePSFSampler(**mas_setup)

In [ ]:
# Generate PSFs no phase errors or atmosphere passed in, ideal case
xh, yh, strehls = mas_psf_sampler.sample(meas_strehl=True)
psf_filter1 = xh[..., 0]
psf_filter2 = xh[..., 1]

plt.figure(figsize=[16, 6])

plt.subplot(121)
im = plt.imshow(np.log10(psf_filter1), vmax=0)
plt.title(f'Strehl {strehls[0]:.03f}')
cbar = plt.colorbar(im)

plt.subplot(122)
im = plt.imshow(np.log10(psf_filter2), vmax=0)
plt.title(f'Strehl {strehls[1]:.03f}')
cbar = plt.colorbar(im)

# For nice fixed plotting range later
log_min = min( np.log10(psf_filter1).min(), np.log10(psf_filter2).min() )

In [ ]:
# Generate some random PTT errors and generate PSFS
errs = np.random.normal(0, .1, (nMir, 3))

xh, yh, strehls = mas_psf_sampler.sample(errs, meas_strehl=True)
psf_filter1 = xh[..., 0]
psf_filter2 = xh[..., 1]

plt.figure(figsize=[16, 6])

plt.subplot(121)
im = plt.imshow(np.log10(psf_filter1), vmax=0, vmin=log_min)
plt.title(f'Strehl {strehls[0]:.03f}')
cbar = plt.colorbar(im)

plt.subplot(122)
im = plt.imshow(np.log10(psf_filter2), vmax=0, vmin=log_min)
plt.title(f'Strehl {strehls[1]:.03f}')
cbar = plt.colorbar(im)

# For nice fixed plotting range later
log_min = min( np.log10(psf_filter1).min(), np.log10(psf_filter2).min() )

### Generate some atmospheres

Pick either multi-layer or single-layer

In [ ]:
fried_params = .25, .550e-6  # r0 (meters), wavelength measured at (meters)
outer_scale = 200            # (meters)

In [ ]:
# Multi-layer atmosphere
layers = hcipy.make_standard_atmospheric_layers(mas_psf_sampler.pupil_grid, outer_scale)
atmos = hcipy.MultiLayerAtmosphere(layers, scintilation=False)
atmos.Cn_squared = hcipy.Cn_squared_from_fried_parameter(*fried_params)
atmos.reset()

In [ ]:
# Single layer atmosphere
cn2 = hcipy.Cn_squared_from_fried_parameter(*fried_params)
atmos = hcipy.InfiniteAtmosphericLayer(mas_psf_sampler.pupil_grid, cn2, outer_scale, 10, 100)

In [ ]:
# Plot a phase screen at 1 micron
atmos_1mic_phase = atmos.phase_for(1e-6)
plt.imshow(atmos_1mic_phase.reshape((256, 256))) # With multi-atmos
plt.colorbar()

### Generate PSFs with atmosphere and no extra PTT errors

Note: When atmosphere is passed in, optimal PTT correction is fit to the pupil and returned sclaed to actuation scale set in sampler (piston_scale and tip_tilt_scale)

In [ ]:
# Plot pupil and PSF with this atmosphere
x, y_fit_atmos, strehls = mas_psf_sampler.sample(atmos=atmos, meas_strehl=True)
psf_filter1 = x[..., 0]
psf_filter2 = x[..., 1]

plt.figure(figsize=[12,10])

# Getting the phase screens to plot isn't as pretty as I'd like
plt.subplot(221)
awf1 = (atmos(mas_psf_sampler.sm(mas_psf_sampler.lam_setups[0]['wfs'][0])))
hcipy.imshow_field(awf1.phase, mask=mas_psf_sampler.aper, cmap="twilight_shifted", vmin=-np.pi, vmax=np.pi)
plt.colorbar()

plt.subplot(222)
awf1 = (atmos(mas_psf_sampler.sm(mas_psf_sampler.lam_setups[1]['wfs'][0])))
hcipy.imshow_field(awf1.phase, mask=mas_psf_sampler.aper, cmap="twilight_shifted", vmin=-np.pi, vmax=np.pi)
plt.colorbar()

plt.subplot(223)
im = plt.imshow(np.log10(psf_filter1), vmax=0, vmin=log_min, cmap='inferno')
plt.title(f'Strehl {strehls[0]:.03f}')
cbar = plt.colorbar(im)

plt.subplot(224)
im = plt.imshow(np.log10(psf_filter2), vmax=0, vmin=log_min, cmap='inferno')
plt.title(f'Strehl {strehls[1]:.03f}')
cbar = plt.colorbar(im)

Do the same again, but feed in the optimal PTTs fit in last call to the sampler

In [ ]:
# Plot pupil and PSF with atmosphere and PTT correction
x, y, strehls = mas_psf_sampler.sample(-y_fit_atmos, atmos=atmos, meas_strehl=True)
psf_filter1 = x[..., 0]
psf_filter2 = x[..., 1]

plt.figure(figsize=[12,10])

# Getting the phase screens to plot isn't as pretty as I'd like
plt.subplot(221)
awf1 = (atmos(mas_psf_sampler.sm(mas_psf_sampler.lam_setups[0]['wfs'][0])))
hcipy.imshow_field(awf1.phase, mask=mas_psf_sampler.aper, cmap="twilight_shifted", vmin=-np.pi, vmax=np.pi)
plt.colorbar()

plt.subplot(222)
awf1 = (atmos(mas_psf_sampler.sm(mas_psf_sampler.lam_setups[1]['wfs'][0])))
hcipy.imshow_field(awf1.phase, mask=mas_psf_sampler.aper, cmap="twilight_shifted", vmin=-np.pi, vmax=np.pi)
plt.colorbar()

plt.subplot(223)
im = plt.imshow(np.log10(psf_filter1), vmax=0, vmin=log_min, cmap='inferno')
plt.title(f'Strehl {strehls[0]:.03f}')
cbar = plt.colorbar(im)

plt.subplot(224)
im = plt.imshow(np.log10(psf_filter2), vmax=0, vmin=log_min, cmap='inferno')
plt.title(f'Strehl {strehls[1]:.03f}')
cbar = plt.colorbar(im)

### Convolve PSF with an image 

If you pass in an image to conolve with PSF, sampler returns that instead of PSFs.  Any extra_processing specified in sampler (intensity power scaling, adding noise, generating FFTs) is applied to these images instead of the PSF

Note: Image's angular pixel scale should agree with scales set in filter configs

Also note: I haven't done anything to scale the output of convolution yet

In [ ]:
orig_im = plt.imread('unnamed4.png')

In [ ]:
# Plot images with atmosphere (no correction)
x, y_fit_atmos, strehls = mas_psf_sampler.sample(atmos=atmos, convolve_im=orig_im, meas_strehl=True)
im_filter1 = x[..., 0]
im_filter2 = x[..., 1]

plt.figure(figsize=[12,10])

# Getting the phase screens to plot isn't as pretty as I'd like
plt.subplot(221)
awf1 = (atmos(mas_psf_sampler.sm(mas_psf_sampler.lam_setups[0]['wfs'][0])))
hcipy.imshow_field(awf1.phase, mask=mas_psf_sampler.aper, cmap="twilight_shifted", vmin=-np.pi, vmax=np.pi)
plt.colorbar()

plt.subplot(222)
awf1 = (atmos(mas_psf_sampler.sm(mas_psf_sampler.lam_setups[1]['wfs'][0])))
hcipy.imshow_field(awf1.phase, mask=mas_psf_sampler.aper, cmap="twilight_shifted", vmin=-np.pi, vmax=np.pi)
plt.colorbar()

plt.subplot(223)
im = plt.imshow(im_filter1, cmap='inferno')
plt.title(f'Strehl {strehls[0]:.03f}')
cbar = plt.colorbar(im)

plt.subplot(224)
im = plt.imshow(im_filter2, cmap='inferno')
plt.title(f'Strehl {strehls[1]:.03f}')
cbar = plt.colorbar(im)

In [ ]:
# Plot images with atmosphere (with correction)
x, y, strehls = mas_psf_sampler.sample(-y_fit_atmos, atmos=atmos, convolve_im=orig_im, meas_strehl=True)
im_filter1 = x[..., 0]
im_filter2 = x[..., 1]

plt.figure(figsize=[12,10])

# Getting the phase screens to plot isn't as pretty as I'd like
plt.subplot(221)
awf1 = (atmos(mas_psf_sampler.sm(mas_psf_sampler.lam_setups[0]['wfs'][0])))
hcipy.imshow_field(awf1.phase, mask=mas_psf_sampler.aper, cmap="twilight_shifted", vmin=-np.pi, vmax=np.pi)
plt.colorbar()

plt.subplot(222)
awf1 = (atmos(mas_psf_sampler.sm(mas_psf_sampler.lam_setups[1]['wfs'][0])))
hcipy.imshow_field(awf1.phase, mask=mas_psf_sampler.aper, cmap="twilight_shifted", vmin=-np.pi, vmax=np.pi)
plt.colorbar()

plt.subplot(223)
im = plt.imshow(im_filter1, cmap='inferno')
plt.title(f'Strehl {strehls[0]:.03f}')
cbar = plt.colorbar(im)

plt.subplot(224)
im = plt.imshow(im_filter2, cmap='inferno')
plt.title(f'Strehl {strehls[1]:.03f}')
cbar = plt.colorbar(im)